In [1]:
import numpy as np
from scipy.optimize import curve_fit
from astropy.table import Table, hstack
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tqdm.notebook as tqdm
import pickle
import os
import paths

In [2]:
data_path = paths.data / "Var_notVar_data"
if not os.path.exists(data_path):
    os.makedirs(data_path)

In [22]:
VarStar_properties1 = Table.read(paths.static / "TDSS_SES+PREV_DR16DR12griLT20_GaiaEDR3_Drake2014PerVar_CSSID_ZTFIDs_LCpointer_PyHammer_EqW_LCProps.fits")
VarStar_properties2 = Table.read(paths.static / "VarStar_LCSTATS2.fits")
VarStar_properties3 = Table.read(paths.static / "VarStar_rawSigmaAboveBelow.fits")

VarStar_properties = hstack([VarStar_properties1, VarStar_properties2, VarStar_properties3])
VarStar_properties.remove_columns(['ra_2', 'dec_2', 'ra_3', 'dec_3'])

control_properties1 = Table.read(paths.static / "VarStar_Controljittered_LCSTATS_withP2.fits")
control_properties2 = Table.read(paths.static / "VarStar_Controljittered_rawSigmaAboveBelow.fits")

control_properties = hstack([control_properties1, control_properties2])

In [23]:
percent_cut = 95
########################################################
########################################################
########################################################

control_holding_g = np.zeros((76, 29)) * np.nan
Varstar_holding_g = np.zeros((76, 29)) * np.nan

pos_steps = np.arange(3.0, 10.25, 0.25)
steps = np.stack((pos_steps*-1, pos_steps)).flatten()
steps.sort()

this_filter = "g"

for ii, ss in enumerate(steps):
    pos_ii = np.where(pos_steps == np.abs(ss))[0][0]
    for jj in control_properties2[f'ZTF_{this_filter}_sigma{ss}'].data.data:
        if np.isfinite(jj):
            if int(jj)==0:
                pass
            else:
                if np.isfinite(control_holding_g[int(jj), pos_ii]):
                    control_holding_g[int(jj), pos_ii] += 1
                else:
                    control_holding_g[int(jj), pos_ii] = 1

for ii, ss in enumerate(steps):
    pos_ii = np.where(pos_steps == np.abs(ss))[0][0]
    for jj in VarStar_properties3[f'ZTF_{this_filter}_sigma{ss}'].data.data:
        if np.isfinite(jj):
            if int(jj)==0:
                pass
            else:
                if np.isfinite(Varstar_holding_g[int(jj), pos_ii]):
                    Varstar_holding_g[int(jj), pos_ii] += 1
                else:
                    Varstar_holding_g[int(jj), pos_ii] = 1
    
percent_line=[]

for ii in range(29):
    #percent_Nstars = np.nanpercentile(control_holding[:,ii]*np.arange(61), percent_cut)
    #print(np.where(control_holding[:,ii]*np.arange(61) <= percent_Nstars)[0][0], np.where(control_holding[:,ii]*np.arange(61) >= np.nanpercentile(control_holding[:,ii]*np.arange(61), percent_cut))[0][-1]+0.5)
    #percent_line.append(np.where(control_holding[:,ii]*np.arange(61) <= percent_Nstars)[0][0])
    #percent_line.append(np.where(control_holding[:,ii]*np.arange(61) >= np.nanpercentile(control_holding[:,ii]*np.arange(61), percent_cut))[0][-1]+0.5)
    
    all_events = []
    for jj in range(control_holding_g.shape[0]):
        if np.isfinite(control_holding_g[jj,ii]):
            all_events.extend([jj]*int(control_holding_g[jj,ii]))
    
    percent_line.append(np.percentile(all_events, percent_cut))
    
percent_line_g = np.array(percent_line)

def f(x, A, B, C):
    return A + B*np.exp(-1*C*x)

popt_g, pcov = curve_fit(f, pos_steps, percent_line_g)
popt = popt_g
########################################################
########################################################
########################################################
########################################################
########################################################

control_holding_r = np.zeros((76, 29)) * np.nan
Varstar_holding_r = np.zeros((76, 29)) * np.nan

pos_steps = np.arange(3.0, 10.25, 0.25)

this_filter = "r"

for ii, ss in enumerate(steps):
    pos_ii = np.where(pos_steps == np.abs(ss))[0][0]
    for jj in control_properties2[f'ZTF_{this_filter}_sigma{ss}'].data.data:
        if np.isfinite(jj):
            if int(jj)==0:
                pass
            else:
                if np.isfinite(control_holding_r[int(jj), pos_ii]):
                    control_holding_r[int(jj), pos_ii] += 1
                else:
                    control_holding_r[int(jj), pos_ii] = 1

for ii, ss in enumerate(steps):
    pos_ii = np.where(pos_steps == np.abs(ss))[0][0]
    for jj in VarStar_properties3[f'ZTF_{this_filter}_sigma{ss}'].data.data:
        if np.isfinite(jj):
            if int(jj)==0:
                pass
            else:
                if np.isfinite(Varstar_holding_r[int(jj), pos_ii]):
                    Varstar_holding_r[int(jj), pos_ii] += 1
                else:
                    Varstar_holding_r[int(jj), pos_ii] = 1
    
percent_line=[]

for ii in range(29):
    #percent_Nstars = np.nanpercentile(control_holding[:,ii]*np.arange(61), percent_cut)
    #print(np.where(control_holding[:,ii]*np.arange(61) <= percent_Nstars)[0][0], np.where(control_holding[:,ii]*np.arange(61) >= np.nanpercentile(control_holding[:,ii]*np.arange(61), percent_cut))[0][-1]+0.5)
    #percent_line.append(np.where(control_holding[:,ii]*np.arange(61) <= percent_Nstars)[0][0])
    #percent_line.append(np.where(control_holding[:,ii]*np.arange(61) >= np.nanpercentile(control_holding[:,ii]*np.arange(61), percent_cut))[0][-1]+0.5)
    
    all_events = []
    for jj in range(control_holding_r.shape[0]):
        if np.isfinite(control_holding_r[jj,ii]):
            all_events.extend([jj]*int(control_holding_r[jj,ii]))
    
    percent_line.append(np.percentile(all_events, percent_cut))
    
percent_line_r = np.array(percent_line)

def f(x, A, B, C):
    return A + B*np.exp(-1*C*x)

popt_r, pcov = curve_fit(f, pos_steps, percent_line_r)
popt = popt_r
########################################################

data = {'percent_cut': percent_cut,
        'control_holding_g': control_holding_g,
        'control_holding_r': control_holding_r,
        'Varstar_holding_g': Varstar_holding_g,
        'Varstar_holding_r': Varstar_holding_r,
        'pos_steps': pos_steps,
        'percent_line_g': percent_line_g,
        'percent_line_r': percent_line_r}

# data

with open(data_path / "Xsigma_Nevents_data.pkl", "wb") as f:
    pickle.dump(data, f)

In [24]:
with open(data_path / "Xsigma_Nevents_data.pkl", 'rb') as fp:
        data = pickle.load(fp)

percent_cut = data['percent_cut']
control_holding_g = data['control_holding_g']
control_holding_r = data['control_holding_r']
Varstar_holding_g = data['Varstar_holding_g']
Varstar_holding_r = data['Varstar_holding_r']
pos_steps = data['pos_steps']
percent_line_g = data['percent_line_g']
percent_line_r = data['percent_line_r']

In [25]:
steps = np.stack((np.arange(3.0, 10.25, 0.25)*-1, np.arange(3.0, 10.25, 0.25))).flatten()
steps.sort()

pos_steps = np.arange(3.0, 10.25, 0.25)

total_truth_g = np.zeros(len(VarStar_properties3))
total_truth_r = np.zeros(len(VarStar_properties3))

sigma_limit = 3.0
N_factor = 5.0
for ii, ROW in enumerate(tqdm.tqdm(VarStar_properties3)):
    hold_truth_g = []
    hold_truth_r = []
    for ss in steps:
        #hold_truth_g.append(ROW[f'ZTF_g_sigma{ss}'] >= f(np.abs(ss), *popt_g)) #using the fit function f
        #hold_truth_r.append(ROW[f'ZTF_r_sigma{ss}'] >= f(np.abs(ss), *popt_r)) #using the fit function f
        if np.abs(ss) < sigma_limit:
            pass
        else:
            hold_truth_g.append(ROW[f'ZTF_g_sigma{ss}'] >=  N_factor * percent_line_g[(pos_steps==np.abs(ss))][0])
            hold_truth_r.append(ROW[f'ZTF_r_sigma{ss}'] >=  N_factor * percent_line_r[(pos_steps==np.abs(ss))][0])

    total_truth_g[ii] = np.any(hold_truth_g)
    total_truth_r[ii] = np.any(hold_truth_r)

  0%|          | 0/25121 [00:00<?, ?it/s]

In [26]:
NaN = np.nan
cut_percentiles = np.array([80, 90, 95, 99])
mulit_stat_selection_index = np.array([3, 2, 1, NaN])  # sum(eq) > this index, ie. if [1] means >=2 stats,[2] mean >=3 stats etc
multi_stat_selection_logFAPlt3_index = np.array([2, 1, NaN, NaN])

print(f"Using {cut_percentiles=}")
print(f"")
print(f"Var if multistat >= {mulit_stat_selection_index} for each cut_percentiles")
print(f"Var if multistat >= {multi_stat_selection_logFAPlt3_index} for each cut_percentiles AND logFAP <= -3")
print(f"Var if logFAP <= -5")
print(f"Var if N_abovebelow_Xsigma > 3*Nfit")
print(f"")

ZTF_g_Chi2_85 =       np.nanpercentile(control_properties['ZTF_g_Chi2'].data.data, cut_percentiles)
ZTF_g_StetJ_85 = np.nanpercentile(control_properties['ZTF_g_stetson_j'].data.data, cut_percentiles)
ZTF_g_VarStat_85 = np.nanpercentile(control_properties['ZTF_g_VarStat'].data.data, cut_percentiles)

ZTF_r_Chi2_85 =       np.nanpercentile(control_properties['ZTF_r_Chi2'].data.data, cut_percentiles)
ZTF_r_StetJ_85 = np.nanpercentile(control_properties['ZTF_r_stetson_j'].data.data, cut_percentiles)
ZTF_r_VarStat_85 = np.nanpercentile(control_properties['ZTF_r_VarStat'].data.data, cut_percentiles)


ZTF_g_Chi2_compare =       (VarStar_properties['ZTF_g_Chi2'] * np.ones((len(ZTF_g_Chi2_85), len(VarStar_properties)))).T.data.data >= (ZTF_g_Chi2_85 * np.ones((len(VarStar_properties), len(ZTF_g_Chi2_85))))
ZTF_g_StetJ_compare = (VarStar_properties['ZTF_g_stetson_j'] * np.ones((len(ZTF_g_Chi2_85), len(VarStar_properties)))).T.data.data >= (ZTF_g_Chi2_85 * np.ones((len(VarStar_properties), len(ZTF_g_Chi2_85))))
ZTF_g_VarStat_compare = (VarStar_properties['ZTF_g_VarStat'] * np.ones((len(ZTF_g_Chi2_85), len(VarStar_properties)))).T.data.data >= (ZTF_g_Chi2_85 * np.ones((len(VarStar_properties), len(ZTF_g_Chi2_85))))

ZTF_r_Chi2_compare =       (VarStar_properties['ZTF_r_Chi2'] * np.ones((len(ZTF_r_Chi2_85), len(VarStar_properties)))).T.data.data >= (ZTF_r_Chi2_85 * np.ones((len(VarStar_properties), len(ZTF_r_Chi2_85))))
ZTF_r_StetJ_compare = (VarStar_properties['ZTF_r_stetson_j'] * np.ones((len(ZTF_g_Chi2_85), len(VarStar_properties)))).T.data.data >= (ZTF_g_Chi2_85 * np.ones((len(VarStar_properties), len(ZTF_g_Chi2_85))))
ZTF_r_VarStat_compare = (VarStar_properties['ZTF_r_VarStat'] * np.ones((len(ZTF_g_Chi2_85), len(VarStar_properties)))).T.data.data >= (ZTF_g_Chi2_85 * np.ones((len(VarStar_properties), len(ZTF_g_Chi2_85))))

g_selection = ZTF_g_Chi2_compare.astype(int) + ZTF_g_StetJ_compare.astype(int) + ZTF_g_VarStat_compare.astype(int)
r_selection = ZTF_r_Chi2_compare.astype(int) + ZTF_r_StetJ_compare.astype(int) + ZTF_r_VarStat_compare.astype(int)

gr_stats_selection = g_selection.astype(int) + r_selection.astype(int)


multi_percent_selection = gr_stats_selection >= (mulit_stat_selection_index * np.ones((len(VarStar_properties), len(cut_percentiles))))
multi_index = np.any(multi_percent_selection, axis=1)


logFAP_lt5_cut = ((VarStar_properties['ZTF_g_logProb'] <= -5.0) | (VarStar_properties['ZTF_r_logProb'] <= -5.0))


logFAP_lt3_cut = ((VarStar_properties['ZTF_g_logProb'] <= -3.0) | (VarStar_properties['ZTF_r_logProb'] <= -3.0)) & np.any(gr_stats_selection >= (multi_stat_selection_logFAPlt3_index * np.ones((len(VarStar_properties), len(cut_percentiles)))), axis=1)

total_truth = (total_truth_g.astype(bool) | total_truth_r.astype(bool))

Nepoch_cut = ((VarStar_properties['ZTF_g_Nepochs'].data.data > 10) | (VarStar_properties['ZTF_r_Nepochs'].data.data > 10))

all_cut = ((multi_index | logFAP_lt5_cut | logFAP_lt3_cut | total_truth) & Nepoch_cut).data
# all_cut.sum()
full_index = all_cut



print(f"Number in {multi_index.sum()=}")
print(f"Number in {logFAP_lt5_cut.data.sum()=}")
print(f"Number in {logFAP_lt3_cut.data.sum()=}")
print()
print(f"Number in BOTH multi_index and logFAP_lt5_cut = {(multi_index & logFAP_lt5_cut.data).sum()}")
print(f"Number in BOTH multi_index and logFAP_lt3_cut = {(multi_index & logFAP_lt3_cut.data).sum()}")
print(f"Number in BOTH logFAP_lt5_cut and logFAP_lt3_cut = {( logFAP_lt5_cut.data & logFAP_lt3_cut.data).sum()}")
print()
print(f"Number in multi_index NOT in (logFAP_lt5_cut OR logFAP_lt3_cut) = {(multi_index & ~(logFAP_lt5_cut.data | logFAP_lt3_cut.data)).sum()}")
print(f"Number in logFAP_lt5_cut NOT in (multi_index OR logFAP_lt3_cut) = {(logFAP_lt5_cut.data & ~(multi_index | logFAP_lt3_cut.data)).sum()}")
print(f"Number in logFAP_lt3_cut NOT in (multi_index OR logFAP_lt5_cut) = {(logFAP_lt3_cut.data & ~(logFAP_lt5_cut.data | multi_index)).sum()}")


logFAP_limit = -5
N_g_periodic = (VarStar_properties[full_index]['ZTF_g_logProb'] <= logFAP_limit).data.sum()
N_r_periodic = (VarStar_properties[full_index]['ZTF_r_logProb'] <= logFAP_limit).data.sum()
N_g_or_r_periodic = ((VarStar_properties[full_index]['ZTF_g_logProb'] <= logFAP_limit).data | (VarStar_properties[full_index]['ZTF_r_logProb'] <= -5).data).sum()
N_g_and_r_periodic = ((VarStar_properties[full_index]['ZTF_g_logProb'] <= logFAP_limit).data & (VarStar_properties[full_index]['ZTF_r_logProb'] <= -5).data).sum()

print()
print(f"{logFAP_limit=}")
print(f"Number periodic in g =  {N_g_periodic} ({np.round(100*N_g_periodic/full_index.sum(), 1)})%")
print(f"Number periodic in r =  {N_r_periodic} ({np.round(100*N_r_periodic/full_index.sum(), 1)})%")
print(f"Number periodic in g OR r =  {N_g_or_r_periodic} ({np.round(100*N_g_or_r_periodic/full_index.sum(), 1)})%")
print(f"Number periodic in g AND r =  {N_g_and_r_periodic} ({np.round(100*N_g_and_r_periodic/full_index.sum(), 1)})%")

VarStar_properties[full_index].write(data_path / "TDSS_VarStar_FINAL_Var_ALL_PROP_STATS.fits", format='fits', overwrite=True)
VarStar_properties[~full_index].write(data_path / "TDSS_VarStar_FINAL_NonVar_ALL_PROP_STATS.fits", format='fits', overwrite=True)

full_index_table = Table()
full_index_table.add_column(full_index, name='full_index')
full_index_table.add_column(full_index.cumsum(), name='cumsum')
full_index_table.write(data_path / "full_index_table.fits", format='fits', overwrite=True)

cut_percentile = 'multi'
print()
print(f"Using a {cut_percentile}% percentile cut leaves {full_index.sum()} objects as variable ({np.round(100*(full_index.sum() / len(full_index)), 1)})%, removing {len(full_index)-full_index.sum()} objects ({np.round(100*((len(full_index)-full_index.sum()) / len(full_index)), 1)})%.")

Using cut_percentiles=array([80, 90, 95, 99])

Var if multistat >= [ 3.  2.  1. nan] for each cut_percentiles
Var if multistat >= [ 2.  1. nan nan] for each cut_percentiles AND logFAP <= -3
Var if logFAP <= -5
Var if N_abovebelow_Xsigma > 3*Nfit

Number in multi_index.sum()=4334
Number in logFAP_lt5_cut.data.sum()=4098
Number in logFAP_lt3_cut.data.sum()=4786

Number in BOTH multi_index and logFAP_lt5_cut = 3159
Number in BOTH multi_index and logFAP_lt3_cut = 3657
Number in BOTH logFAP_lt5_cut and logFAP_lt3_cut = 3439

Number in multi_index NOT in (logFAP_lt5_cut OR logFAP_lt3_cut) = 677
Number in logFAP_lt5_cut NOT in (multi_index OR logFAP_lt3_cut) = 659
Number in logFAP_lt3_cut NOT in (multi_index OR logFAP_lt5_cut) = 849

logFAP_limit=-5
Number periodic in g =  3153 (27.1)%
Number periodic in r =  3658 (31.4)%
Number periodic in g OR r =  4098 (35.2)%
Number periodic in g AND r =  2713 (23.3)%

Using a multi% percentile cut leaves 11654 objects as variable (46.4)%, removing 13467 